In [1]:
import os
from pathlib import Path

store_dir = "/store/kruu/eye_tracking/training_data"

****
# Whole task detection
****

In [2]:
from utils.helper import load_and_process_et

features = ['Recording timestamp [ms]', 'epoch_ms', 'Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Event']
interpolate_cols = ['Gaze point X [DACS px]', 'Gaze point Y [DACS px]']
fill_columns = ['Gaze point X [DACS px]', 'Gaze point Y [DACS px]']

chunks_et_without_filtering, blinks_without_filtering, atco_task_map = load_and_process_et(root_dir = store_dir,
                                                            columns = features,
                                                            filter_outliers = False,
                                                            interpolate_cols = interpolate_cols,
                                                            fill_cols = fill_columns, 
                                                            time_resampling=False,)

chunks_et_with_filtering, blinks_with_filtering, atco_task_map = load_and_process_et(root_dir = store_dir,
                                                            columns = features,
                                                            filter_outliers = True,
                                                            interpolate_cols = interpolate_cols,
                                                            fill_cols = fill_columns, 
                                                            time_resampling=False,)

KeyboardInterrupt: 

In [4]:
atco_task_map

{'Aircraft requests': 'Task 0',
 'Assume': 'Task 1',
 'Conflict resolution': 'Task 2',
 'Entry conditions': 'Task 3',
 'Entry conflict resolution': 'Task 4',
 'Entry coordination': 'Task 5',
 'Exit conditions': 'Task 6',
 'Exit conflict resolution': 'Task 7',
 'Exit coordination': 'Task 8',
 'Non-conformance resolution': 'Task 9',
 'QoS': 'Task 10',
 'Return to route': 'Task 11',
 'Transfer': 'Task 12',
 'Zone conflict': 'Task 13'}

In [5]:
# For whole tasks

from collections import defaultdict
import numpy as np

durations_by_task = defaultdict(list)
task_count = defaultdict(int)

for df in chunks_et_without_filtering.values():
    task_id = int(df["Task_id"].iloc[0])
    task_count[task_id] += 1

    ts_col = "Recording timestamp [ms]"
    ts = df[ts_col].astype(float)
    dur_ms = float(ts.max() - ts.min())

    durations_by_task[task_id].append(dur_ms)

mean_ms_by_task = {task: float(np.mean(durs)) for task, durs in durations_by_task.items()}
med_ms_by_task = {task: float(np.median(durs)) for task, durs in durations_by_task.items()}
max_ms_by_task = {task: float(np.max(durs)) for task, durs in durations_by_task.items()}
min_ms_by_task = {task: float(np.min(durs)) for task, durs in durations_by_task.items()}

# The mean and median should be close. Otherwise we have a lot of outliers
for task_id in sorted(max_ms_by_task):
    print(f"Task {task_id}: {mean_ms_by_task[task_id]/1000:.3f}s, {med_ms_by_task[task_id]/1000:.3f}s [{min_ms_by_task[task_id]/1000:.3f}, {max_ms_by_task[task_id]/1000:.3f}] s")
    
print("\n")
print(task_count)

Task 0: 15.254s, 12.438s [0.821, 94.872] s
Task 1: 8.351s, 7.177s [0.371, 492.926] s
Task 2: 18.405s, 9.908s [2.059, 1340.954] s
Task 3: 18.934s, 18.034s [2.226, 62.712] s
Task 4: 19.955s, 19.255s [1.582, 62.712] s
Task 5: 18.848s, 17.352s [1.021, 105.677] s
Task 6: 12.504s, 9.273s [1.232, 70.116] s
Task 7: 13.627s, 7.894s [1.628, 53.879] s
Task 8: 18.497s, 16.043s [1.705, 76.243] s
Task 9: 15.843s, 13.396s [4.493, 40.152] s
Task 10: 8.332s, 7.362s [0.540, 32.903] s
Task 11: 9.328s, 8.302s [0.011, 44.322] s
Task 12: 11.216s, 7.862s [0.004, 3284.167] s
Task 13: 12.221s, 9.969s [1.011, 302.949] s


defaultdict(<class 'int'>, {5: 293, 1: 1753, 13: 752, 9: 50, 0: 313, 2: 301, 12: 1720, 11: 508, 8: 209, 6: 123, 4: 25, 10: 334, 3: 63, 7: 16})


In [6]:
durations_by_task = defaultdict(list)
task_count = defaultdict(int)

for df in chunks_et_with_filtering.values():
    task_id = int(df["Task_id"].iloc[0])
    task_count[task_id] += 1

    ts_col = "Recording timestamp [ms]"
    ts = df[ts_col].astype(float)
    dur_ms = float(ts.max() - ts.min())

    durations_by_task[task_id].append(dur_ms)

mean_ms_by_task = {task: float(np.mean(durs)) for task, durs in durations_by_task.items()}
med_ms_by_task = {task: float(np.median(durs)) for task, durs in durations_by_task.items()}
max_ms_by_task = {task: float(np.max(durs)) for task, durs in durations_by_task.items()}
min_ms_by_task = {task: float(np.min(durs)) for task, durs in durations_by_task.items()}

# The mean and median should be close. Otherwise we have a lot of outliers
for task_id in sorted(max_ms_by_task):
    print(f"Task {task_id}: {mean_ms_by_task[task_id]/1000:.3f}s, {med_ms_by_task[task_id]/1000:.3f}s [{min_ms_by_task[task_id]/1000:.3f}, {max_ms_by_task[task_id]/1000:.3f}] s")
    
print("\n")

print(task_count)

Task 0: 14.837s, 12.330s [0.821, 53.370] s
Task 1: 7.727s, 7.106s [0.715, 26.745] s
Task 2: 11.221s, 9.603s [2.059, 36.805] s
Task 3: 18.228s, 18.005s [2.226, 47.518] s
Task 4: 19.955s, 19.255s [1.582, 62.712] s
Task 5: 18.075s, 17.317s [1.021, 47.518] s
Task 6: 11.502s, 9.184s [1.232, 37.135] s
Task 7: 10.944s, 7.773s [1.628, 32.910] s
Task 8: 17.731s, 15.836s [1.705, 52.108] s
Task 9: 15.843s, 13.396s [4.493, 40.152] s
Task 10: 8.192s, 7.322s [0.540, 26.931] s
Task 11: 9.062s, 8.257s [0.787, 30.014] s
Task 12: 8.109s, 7.801s [0.540, 25.344] s
Task 13: 10.435s, 9.764s [1.011, 33.280] s


defaultdict(<class 'int'>, {5: 289, 1: 1732, 13: 737, 9: 50, 0: 311, 2: 290, 12: 1701, 11: 502, 8: 206, 6: 120, 4: 25, 10: 332, 3: 62, 7: 15})


A cleaning step is needed here to remove annotation errors. Added in data_processing_gaze_data.task_range_finder.

Procedure:
- Remove tasks where duration < 0.5s
- Discard bounds that are outside quantile rule:
    - Upper bound = Q3 + 3 * IQR
    - Lower bound = max(0.5s, Q1 - 1.5 * IQR)

****
# New labelling strategy
****

- Prediction at time t:
    - Label = task at time t 
    - define marin around eah task boundary (ex 2/3s) to avoid noise
    - Option: use sample weights to soften boundary samples (e.g. fraction of the last 10s that belongs to the labeled task), and use that purity as a sample weight in XGboost. The model pay less attention to the ambiguous cases. 
    - Option: hack sfot labels in XGboost. Ex window has 70% Task 1, and 30% idle
        - Create two training rows with the same features but with different labels (label Task 1, weight 0.7 / label idle, weight 0.3)
        - Train XGBoost with sample weight. 
        - Gradient contributions approximate soft labels
        - Mix with end-anchored idea: label by task at time t (for evaluation), but distribute training weights according to window composition
- 3 windows:
    - short (3-5s), mid (10-15s) and long (25-40s) terms
        - Short: captures immediate events (clicks, fixatino, saccades)
        - Mid: typical average task length region
        - Long: context
    - Extract features separately on each window (prefix features names like short_*, mid_*, long_*)
    - Concatenate all these features into a single vector.
    - Label the training row wit the task active at time t (or idle if none)
    - "Given what happened in the last S/M/L seconds up to time t, what is the controller doing now?"

In [3]:
from utils.helper import load_and_process_et

# 30 mins

store_dir = "/store/kruu/eye_tracking/training_data"
features = ['Recording timestamp [ms]', 'epoch_ms', 'Gaze point X [DACS px]', 'Gaze point Y [DACS px]', 'Event']
interpolate_cols = ['Gaze point X [DACS px]', 'Gaze point Y [DACS px]']
fill_columns = ['Gaze point X [DACS px]', 'Gaze point Y [DACS px]']

chunks_et, blinks, atco_task_map = load_and_process_et(root_dir = store_dir,
                                                            columns = features,
                                                            interpolate_cols = interpolate_cols,
                                                            fill_cols = fill_columns,
                                                            window_short_ms = 5000,
                                                            window_mid_ms = 10000,
                                                            window_long_ms = 25000,
                                                            task_margin_ms = 2000,
                                                            step_ms = 3000,
                                                            filter_outliers = True,
                                                            time_resampling=False)

⚠️ Unmatched 'end' for Task 5 at 3459514
⚠️ Unmatched 'start' for Task 3 at 3443448
⚠️ Unmatched 'end' for Task 5 at 1475678
⚠️ Unmatched 'end' for Task 3 at 732128
⚠️ Unmatched 'end' for Task 8 at 1168579
⚠️ Unmatched 'end' for Task 1 at 1407071
⚠️ Unmatched 'start' for Task 12 at 1171948
⚠️ Unmatched 'end' for Task 7 at 1407679
⚠️ Unmatched 'end' for Task 8 at 3096761
⚠️ Unmatched 'end' for Task 0 at 3174255
⚠️ Unmatched 'start' for Task 1 at 695652
⚠️ Unmatched 'start' for Task 12 at 3785377
⚠️ Unmatched 'start' for Task 13 at 3467085
⚠️ Unmatched 'start' for Task 13 at 3467602
⚠️ Unmatched 'start' for Task 13 at 3467757
⚠️ Unmatched 'start' for Task 6 at 1395939
⚠️ Unmatched 'start' for Task 4 at 3513371
⚠️ Unmatched 'end' for Task 1 at 2525380
⚠️ Unmatched 'end' for Task 13 at 3253527
⚠️ Unmatched 'start' for Task 12 at 3248347
⚠️ Unmatched 'end' for Task 13 at 2910462
⚠️ Unmatched 'end' for Task 7 at 3179705
⚠️ Unmatched 'end' for Task 1 at 346849
⚠️ Unmatched 'end' for Task 12 a

In [3]:
from collections import Counter, defaultdict

keys = chunks_et.keys()

task_counts = Counter()

for k in keys:
    participant, scenario, task, occurrence = k.split("_")
    task_counts[(participant, scenario, task)] += 1

# Step 2: sum counts per (participant, scenario)
scenario_totals = defaultdict(int)

for (participant, scenario, task), count in task_counts.items():
    scenario_totals[(participant, scenario)] += count

# Step 3: compute proportions
task_proportions = {}

for (participant, scenario, task), count in task_counts.items():
    total = scenario_totals[(participant, scenario)]
    proportion = count / total
    task_proportions[(participant, scenario, task)] = proportion

# Pretty print:
for k, p in task_proportions.items():
    participant, scenario, task = k
    if task == '-1':
        print(f"Participant {participant}, Scenario {scenario}, Task {task} → proportion {p:.3f}")

Participant 001, Scenario 1, Task -1 → proportion 0.642
Participant 001, Scenario 2, Task -1 → proportion 0.584
Participant 001, Scenario 3, Task -1 → proportion 0.769
Participant 002, Scenario 1, Task -1 → proportion 0.783
Participant 002, Scenario 2, Task -1 → proportion 0.725
Participant 002, Scenario 3, Task -1 → proportion 0.741
Participant 003, Scenario 1, Task -1 → proportion 0.727
Participant 003, Scenario 3, Task -1 → proportion 0.648
Participant 004, Scenario 1, Task -1 → proportion 0.780
Participant 004, Scenario 2, Task -1 → proportion 0.802
Participant 004, Scenario 3, Task -1 → proportion 0.812
Participant 005, Scenario 1, Task -1 → proportion 0.842
Participant 005, Scenario 2, Task -1 → proportion 0.823
Participant 005, Scenario 3, Task -1 → proportion 0.769
Participant 006, Scenario 1, Task -1 → proportion 0.695
Participant 006, Scenario 2, Task -1 → proportion 0.702
Participant 007, Scenario 1, Task -1 → proportion 0.781
Participant 007, Scenario 2, Task -1 → proportio

In [5]:
from collections import Counter

print("Number of total occurences per task: ")
task_window_counts = Counter(int(key.split('_')[2]) for key in chunks_et.keys())
for task_id in sorted(task_window_counts):
    print(f"Task {task_id}: {task_window_counts[task_id]} windows")

Number of total occurences per task: 
Task -1: 47080 windows
Task 0: 1457 windows
Task 1: 4432 windows
Task 2: 1041 windows
Task 3: 202 windows
Task 4: 148 windows
Task 5: 1613 windows
Task 6: 351 windows
Task 7: 50 windows
Task 8: 1111 windows
Task 9: 260 windows
Task 10: 858 windows
Task 11: 1456 windows
Task 12: 4577 windows
Task 13: 2481 windows


In [6]:
import importlib
import utils.helper
importlib.reload(utils.helper)
from utils.helper import drop_chunks_with_nan_et

print(len(chunks_et))

cleaned_chunks_et = drop_chunks_with_nan_et(chunks_et, threshold=0.8)

67117
Dropped 4781 multi-scale samples due to a proportion of Nans greater than 0.8 in eye-tracking window(s): ['001_1_-1_13', '001_1_-1_14', '001_1_-1_15', '001_1_-1_16', '001_1_-1_17', '001_1_-1_18', '001_1_-1_19', '001_1_-1_20', '001_1_-1_21', '001_1_-1_22'] ...


In [7]:
print("Number of total occurences per task: ")
task_window_counts = Counter(int(key.split('_')[2]) for key in cleaned_chunks_et.keys())
for task_id in sorted(task_window_counts):
    print(f"Task {task_id}: {task_window_counts[task_id]} windows")

Number of total occurences per task: 
Task -1: 43708 windows
Task 0: 1405 windows
Task 1: 4143 windows
Task 2: 964 windows
Task 3: 175 windows
Task 4: 134 windows
Task 5: 1377 windows
Task 6: 320 windows
Task 7: 46 windows
Task 8: 990 windows
Task 9: 241 windows
Task 10: 826 windows
Task 11: 1416 windows
Task 12: 4167 windows
Task 13: 2424 windows


****
# Feature extraction
****

In [84]:
import utils.data_processing_gaze_data
import utils.data_processing_mouse_data
import utils.data_processing_asd_events
importlib.reload(utils.data_processing_gaze_data)
importlib.reload(utils.data_processing_mouse_data)
importlib.reload(utils.data_processing_asd_events)

from utils.helper import load_asd_scenario_data
from utils.data_processing_gaze_data import GazeMetricsProcessor
from utils.data_processing_mouse_data import MouseMetricsProcessor
from utils.data_processing_asd_events import ASDEventsMetricsProcessor
import pandas as pd
from tqdm import tqdm

# ~2h30

asd_scenarios = load_asd_scenario_data(root_dir=store_dir)

prepared_asd = {}
for key, df_sc in asd_scenarios.items():
    asd = df_sc.sort_values("epoch_ms").set_index("epoch_ms")
    prepared_asd[key] = asd

window_keys = ("short", "mid", "long")
rows = []
i = 0

for uid, windows in tqdm(cleaned_chunks_et.items()):

    row = {"id": uid}
    row["participant_id"] = windows["short"]["Participant name"].iloc[0]
    row["Task_id"] = windows["short"]["Task_id"].iloc[0]
    
    p_s_id = "_".join(uid.split("_")[:2])
    scenario_asd = prepared_asd.get(p_s_id)
    if scenario_asd is None:
        continue

    for wname in window_keys:
        chunk = windows[wname]
        min_epoch = chunk["epoch_ms"].min()
        max_epoch = chunk["epoch_ms"].max()
        # fast time slice thanks to index on epoch_ms
        window_asd = scenario_asd.loc[min_epoch:max_epoch].reset_index()
        
        # Gaze metrics
        gaze_processor = GazeMetricsProcessor(chunk, timestamp_unit="ms")
        gaze_metrics = gaze_processor.compute_all_metrics()   
        prefixed_gaze = {f"{wname}_{k}": v for k, v in gaze_metrics.items()}
        row.update(prefixed_gaze)
        
        # Mouse metrics
        mouse_processor = MouseMetricsProcessor(window_asd, resample=False)
        mouse_metrics = mouse_processor.compute_all_metrics()   
        prefixed_mouse = {f"{wname}_{k}": v for k, v in mouse_metrics.items()}
        row.update(prefixed_mouse)
        
        # ASD events metrics
        asd_processor = ASDEventsMetricsProcessor(window_asd)
        asd_metrics = asd_processor.compute_all_metrics()   
        prefixed_asd = {f"{wname}_{k}": v.iat[0] for k, v in asd_metrics.items()}
        row.update(prefixed_asd)

    rows.append(row)
    i += 1
    if i == 10000:
        break
    
metrics_df = pd.DataFrame(rows)

 16%|█▌        | 9999/62336 [24:03<2:05:56,  6.93it/s] 


# TS Fresh

In [ ]:
import trainings._02_xgboost_hierarchical_training
importlib.reload(trainings._02_xgboost_hierarchical_training)
from trainings._02_xgboost_hierarchical_training import extract_tsfresh_features_from_multiscale_chunks

# 20 mins

tsfresh_data = extract_tsfresh_features_from_multiscale_chunks(
        cleaned_chunks_et, 
        ['Gaze point X [DACS px]', 'Gaze point Y [DACS px]'], 
        pval_threshold=0.05, 
        n_jobs=50)


Extracting TSFresh features...


Feature Extraction: 100%|██████████| 250/250 [00:39<00:00,  6.27it/s]


Extracting TSFresh features...


Feature Extraction: 100%|██████████| 250/250 [00:37<00:00,  6.65it/s]


Extracting TSFresh features...


Feature Extraction: 100%|██████████| 250/250 [00:56<00:00,  4.41it/s]


In [86]:
xgboost_data = metrics_df.merge(tsfresh_data, on="id", how="inner")
xgboost_data

,id,participant_id,Task_id,short_Fixation Count,short_Total Fixation Duration (s),short_Avg Fixation Duration (s),short_Saccade Count,short_Avg Saccade Amplitude (px),short_Avg Saccade Velocity (px/s),short_Avg Gaze Velocity (px/s),...,long_Gaze point X [DACS px]__mean,long_Gaze point X [DACS px]__sum_values,long_Gaze point X [DACS px]__absolute_maximum,long_Gaze point Y [DACS px]__mean,long_Gaze point X [DACS px]__maximum,long_Gaze point Y [DACS px]__sum_values,long_Gaze point Y [DACS px]__standard_deviation,long_Gaze point Y [DACS px]__variance,long_Gaze point X [DACS px]__root_mean_square,long_Gaze point Y [DACS px]__median
0,001_1_-1_0,001,-1,13,3.784,0.291077,48,122.385408,14717.509438,1758.013579,...,725.359487,2320425.0,2057.0,806.727727,2057.0,2580722.0,395.005788,156029.572945,1127.259914,861.0
1,001_1_-1_1,001,-1,17,4.415,0.259706,46,103.709524,12525.211390,1699.833873,...,825.513107,2582205.0,2057.0,774.083440,2057.0,2421333.0,389.472247,151688.630826,1121.652692,854.0
2,001_1_-1_2,001,-1,18,4.142,0.230111,46,107.551738,13021.663369,1852.512615,...,1101.328501,3342532.0,2057.0,601.314662,2057.0,1824990.0,307.172021,94354.650576,1204.952224,592.0
3,001_1_-1_3,001,-1,10,4.232,0.423200,25,107.973174,12993.801937,1348.387791,...,1164.679189,3503355.0,2057.0,612.753989,2057.0,1843164.0,311.393455,96965.883628,1249.838991,641.0
4,001_1_-1_4,001,-1,8,2.715,0.339375,18,131.851105,15933.778276,956.915427,...,1154.649485,3472031.0,2057.0,675.059195,2057.0,2029903.0,324.868404,105539.479702,1245.739248,659.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,004_1_-1_719,004,-1,3,2.508,0.836000,3,77.118507,9243.580881,291.685870,...,824.291331,2557776.0,1381.0,642.872059,1381.0,1994832.0,155.327585,24126.658784,834.419925,678.0
9996,004_1_12_42,004,12,5,3.123,0.624600,21,269.994729,32533.202835,1265.580060,...,852.442639,2674965.0,1510.0,644.019439,1510.0,2020933.0,159.616701,25477.491337,864.659388,677.0
9997,004_1_-1_720,004,-1,3,4.641,1.547000,17,256.284176,31111.362248,1273.070339,...,874.477055,2744109.0,1510.0,628.441364,1510.0,1972049.0,143.129931,20486.177091,884.481787,673.0
9998,004_1_-1_721,004,-1,5,4.533,0.906600,12,118.593590,14094.333843,873.121744,...,867.415870,2721951.0,1510.0,621.753346,1510.0,1951062.0,149.102641,22231.597543,878.166995,671.0


****
# Training set
****

In [87]:
from trainings._02_xgboost_hierarchical_training import split_by_participant

train_df, val_df, test_df, parts = split_by_participant(xgboost_data, group_col="participant_id",
                                                    test_size=0.2, val_size=0.1, random_state=42)